# 実験を終了する

実験記録をデータガバナンス機能に保存して、実験を終了します。

## 1.実行環境構成を記録する

In [ ]:
from util.scripts import utils

# 実験名を取得する
%store -r EXPERIMENT_TITLE

# activate状態のconda環境を特定する
env_list = !conda env list
for env in env_list:
    if '*' in env:
        env = env.split(' ')
        activate_env = env[0]

!conda env export -n $activate_env > ~/experiments/$EXPERIMENT_TITLE/environment.yml
!pip freeze > ~/experiments/$EXPERIMENT_TITLE/requirements.txt

## 2. 実験記録をデータガバナンス機能に同期する

実験記録をデータガバナンス機能に同期します。  
以下のセルを実行してください。

In [ ]:
from IPython.display import display, Javascript
display(Javascript('IPython.notebook.save_checkpoint();'))

In [ ]:
import os
import glob
from util.scripts import utils

# 実験パッケージパスを取得する
experiment_path = '/home/jovyan/experiments/' + EXPERIMENT_TITLE

# Git管理のパスのリストを作成する
%cd $experiment_path
files = os.listdir()
# ディレクトリ一覧からGit-annex管理するディレクトリ(input_dataとoutput_data)を排除する
dirs = [f for f in files if os.path.isdir(f)]
dirs.remove('input_data')
dirs.remove('output_data')
# HOME直下のファイルを取得
files = [f for f in files if os.path.isfile(f)]
# Git管理するパスの配列を作成する
files.extend(dirs)
save_path = []
for file in files:
    save_path.append(experiment_path + '/' + file)

# Git-annex管理するパスの配列を作成する
annexed_save_path = [experiment_path + '/input_data', experiment_path + '/output_data']

以下を実行して、`リポジトリ側の変更と競合しました。競合を解決してください。`と表示された場合は、[こちらのFAQ](http://dg02.dg.rcos.nii.ac.jp/G-Node/Info/wiki/%E3%83%AF%E3%83%BC%E3%82%AF%E3%83%95%E3%83%AD%E3%83%BC#1-1%E5%90%8C%E6%9C%9F%E5%87%A6%E7%90%86%E3%82%92%E5%AE%9F%E8%A1%8C%E3%81%99%E3%82%8B%E3%81%A8%E3%80%81%E3%83%AA%E3%83%9D%E3%82%B8%E3%83%88%E3%83%AA%E5%81%B4%E3%81%AE%E5%A4%89%E6%9B%B4%E3%81%A8%E7%AB%B6%E5%90%88%E3%81%97%E3%81%BE%E3%81%97%E3%81%9F%E3%80%82%E7%AB%B6%E5%90%88%E3%82%92%E8%A7%A3%E6%B1%BA%E3%81%97%E3%81%A6%E3%81%8F%E3%81%A0%E3%81%95%E3%81%84%E3%80%82%E3%81%A8%E8%A1%A8%E7%A4%BA%E3%81%95%E3%82%8C%E3%82%8B)を参考に競合を解決してください。

In [ ]:
import papermill as pm
from colorama import Fore
from IPython.display import clear_output

%cd ~/
# Git-annex管理ファイルを保存
try:
    pm.execute_notebook(
        'WORKFLOWS/EX-WORKFLOWS/util/base_datalad_save_push.ipynb',
        '/home/jovyan/.local/push_log.ipynb',
        parameters = dict(SAVE_MESSAGE = EXPERIMENT_TITLE + '_実験終了 (1/2)', PATH = annexed_save_path, IS_RECURSIVE = False)
    )
finally:
    clear_output()
    %store -r DATALAD_MESSAGE
    %store -r DATALAD_ERROR
    print('\n' + DATALAD_MESSAGE + '\n')
    print(Fore.RED + DATALAD_ERROR)

In [ ]:
import papermill as pm
from colorama import Fore
from IPython.display import clear_output

%cd ~/
# Git管理ファイルを保存
try:
    pm.execute_notebook(
        'WORKFLOWS/EX-WORKFLOWS/util/base_datalad_save_push.ipynb',
        '/home/jovyan/.local/push_log.ipynb',
        parameters = dict(SAVE_MESSAGE = EXPERIMENT_TITLE + '_実験終了 (2/2)', TO_GIT = True, PATH = save_path, IS_RECURSIVE = False)
    )
finally:
    clear_output()
    %store -r DATALAD_MESSAGE
    %store -r DATALAD_ERROR
    print('\n' + DATALAD_MESSAGE + '\n')
    print(Fore.RED + DATALAD_ERROR)

## 3. この実験実行環境を停止・削除する

コード利用機能を利用している場合、不要になったこの実行環境を停止・削除します。  
コード付帯機能では20までしか実行環境を構築できないため、不要な実行環境は削除することを推奨します。

### 3.1 この実行環境を確認する

以下のセルを実行して実行環境のサーバー名を確認して下さい。  

In [ ]:
import os

print(os.environ["JUPYTERHUB_SERVICE_PREFIX"].split('/')[3])

### 3.2 コード付帯機能のコントロールパネルへ遷移し、実行環境を停止・削除する

[コントロールパネル](https://jupyter.cs.rcos.nii.ac.jp/hub/home)へ遷移して、`4.1`で確認したサーバーを`stop`、`delete`ボタンをクリックして停止・削除してください。  
※`delete`ボタンは、以下の図のように`stop`ボタンをクリックした後に表示されます。  
![コンテナ削除キャプチャ](https://raw.githubusercontent.com/NII-DG/workflow-template/develop/sections/PACKAGE/base/EX-WORKFLOW/images/%E3%82%B3%E3%83%B3%E3%83%86%E3%83%8A%E5%89%8A%E9%99%A4%E3%82%AD%E3%83%A3%E3%83%97%E3%83%81%E3%83%A3.png)